In [1]:
# prepare model 
# prepare benign images & adversarial images -> value range should be the same
# extract simple NIC model 

In [2]:
from MNIST_models import *
import PI

meta_params = {
    'num_of_train_dataset': 1000,
    'num_of_test_dataset': 100,
    'is_flatten': False
}


PI = PI.PIInterface(meta_params)
model = load_model('store/MNIST_CNN.pt')
PI.set_model(model)
print('train acc:', PI.eval_model('train'))
print('test acc:', PI.eval_model('test'))

1000 100
(100, 1, 28, 28)


/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'MNIST_models.CNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessi

train acc: 0.991
test acc: 0.98


In [3]:
import pickle
prefix = 'store/'    
# LOAD 
adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
set_of_train_dataset, set_of_test_dataset = [], []

for adv_type in adv_types:
    # extract from the store file 
    if adv_type == 'None': fn_name=prefix+'normal.txt'
    else: fn_name=prefix+adv_type+'.txt'
    fp = open(fn_name, 'rb')
    set_of_signatures = pickle.load(fp)
    
    # separate and store in for later training and evaluation 
    if adv_type == 'None': split_percentage = 0.8
    else: split_percentage = 0.8
    split_line = int(len(set_of_signatures)*split_percentage)
    train_set_of_signatures, test_set_of_signatures = set_of_signatures[:split_line], set_of_signatures[split_line:]
    set_of_train_dataset.append(train_set_of_signatures)
    set_of_test_dataset.append(test_set_of_signatures)
    fp.close()
#     set_of_signatures = np.array(set_of_signatures)
#     for i in range(4):
#         print(np.max(set_of_signatures[0][i][0]), np.min(set_of_signatures[0][i][0]))
    
    print(adv_type, len(set_of_signatures), len(train_set_of_signatures), len(test_set_of_signatures))

None 1000 800 200
FGSM 813 650 163
JSMA 966 772 194
CWL2 877 701 176
LINFPGD 978 782 196
LINFBI 970 776 194
ENL1 1000 800 200
ST 991 792 199


In [4]:
import torch
from torch.autograd import Variable

class NIC(nn.Module):
    def __init__(self):
        super(NIC, self).__init__()
        n = 2
        m = 2

        self.fc_compress_1 = nn.Linear(16*24*24, n)
        self.fc_compress_2 = nn.Linear(16*10*10, n)
        self.fc_compress_3 = nn.Linear(32*3*3, n)
        self.fc_compress_4 = nn.Linear(64, n)

        self.softmax = nn.Softmax()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        
        self.output = nn.Linear(n, m)
        
    def forward(self, x1, x2, x3, x4):
        x1 = x1.view(-1, 16*24*24)
        x2 = x2.view(-1, 16*10*10)
        x3 = x3.view(-1, 32*3*3)
        x4 = x4.view(-1, 64)
        
        x1 = self.relu(self.fc_compress_1(x1))
        x2 = self.relu(self.fc_compress_2(x2))
        x3 = self.relu(self.fc_compress_3(x3))
        x4 = self.relu(self.fc_compress_4(x4))
        
        x1 = self.dropout(x1)
        x2 = self.dropout(x2)
        x3 = self.dropout(x3)
        x4 = self.dropout(x4)

        s = torch.add(x1, x2)
        
        s = self.relu(s)
        s = self.dropout(s)
        
        s = torch.add(s, x3)
        
        s = self.relu(s)
        s = self.dropout(s)
        
        s = torch.add(s, x4)
        
        s = self.relu(s)
        s = self.dropout(s)
        
        s = self.relu(self.output(s))
        
        return self.softmax(s)
    
def test_guard_model(nic, set_of_test_dataset, adv_types, verbose=False):
    nic.eval()
    total_train_correct_count, total_train_count = 0, 0 
    for test_dataset, adv_type in zip(set_of_test_dataset, adv_types):
        current_count = 0
        for singatures in test_dataset:
            f1, f2, f3, f4 = preprocess(singatures)
            outputs = nic.forward(f1, f2, f3, f4)
            if adv_type == 'None': label = torch.from_numpy(np.array([[1, 0]])).float()
            else: label = torch.from_numpy(np.array([[0, 1]])).float()

            prediction = (outputs.max(1, keepdim=True)[1]).item()     
            if adv_type == 'None': 
                if (prediction == 0): 
                    current_count += 1
            else: 
                if (prediction == 1): 
                    current_count += 1
            
        # record the current train set acc
        if verbose:
            if adv_type == 'None': 
                print('benign correct:', current_count, '/', len(test_dataset))
            else:
                print('adv (', adv_type, ') correct:', current_count, '/', len(test_dataset))

        total_train_correct_count += current_count
        total_train_count += len(test_dataset)

    acc = total_train_correct_count/total_train_count
    if verbose:
        print('acc:', acc)
        
        
    nic.train()
    return acc
    
nic = NIC()
nic.train()
optimizer = torch.optim.Adam(nic.parameters())
loss_func = nn.BCELoss()
epoches = 0

train_accs, test_accs, losses = [], [], []
set_train_sub_accs, set_test_sub_accs = [], []

for epoch in range(epoches):
    total_loss = None 
    # labeling ...
    train_dataset, train_labels = [], []
    for dataset, adv_type in zip(set_of_train_dataset, adv_types):
        for singatures in dataset:
            if adv_type == 'None': 
                for _ in range(1):
                    train_dataset.append(singatures)
                    label = torch.from_numpy(np.array([[1, 0]])).float()
                    train_labels.append(label)

            else: 
                train_dataset.append(singatures)
                label = torch.from_numpy(np.array([[0, 1]])).float()
                train_labels.append(label)

    # shuffling 
    shuffle_indexs = np.arange(len(train_dataset))
    np.random.shuffle(shuffle_indexs)

    # training 
    for index in shuffle_indexs:
        singatures, label = train_dataset[index], train_labels[index]
        f1, f2, f3, f4 = preprocess(singatures)
        outputs = nic.forward(f1, f2, f3, f4)

        # for recording the training process 
        loss = loss_func(outputs, label) 
        if total_loss is None: total_loss = loss 
        else: total_loss += loss

        # Optimization (back-propogation)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch:', (epoch+1), 'loss:', total_loss.item())    
    train_acc = test_guard_model(nic, set_of_train_dataset, adv_types, verbose=True)
    test_acc = test_guard_model(nic, set_of_test_dataset, adv_types, verbose=True)
    print('train (acc):', train_acc)
    print('test (acc):', test_acc)
    print()
    




In [5]:
# GENERATE & STORE
# adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
# for adv_type in adv_types:
#     if adv_type == 'None': 
#         set_of_signatures = PI.generate_signatures() 
#     else: 
#         fn_name='store/'+adv_type+'.txt'
#         set_of_signatures = PI.generate_signatures(adv_type=adv_type)

# LOAD
prefixs = ['store_zero/', 'store_one/', 'store_two/', 'store_three/', 'store_four/', 
        'store_five/', 'store_six/', 'store_seven/', 'store_eight/', 'store_nine/']    

one_to_nine_set_of_train_dataset = []
one_to_nine_set_of_test_dataset = []

for idx, prefix in enumerate(prefixs):
    fn = 'store_subs_fadv/'+prefix+'guard.pt'
    
    set_of_train_dataset = []
    set_of_test_dataset = []
    
    # LOAD DATASET 
    adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
    set_of_train_dataset, set_of_test_dataset = [], []
    for adv_type in adv_types:
        
        set_of_signatures = []
        for i in range(1000):
            # extract from the store file 
            if adv_type == 'None': fn_name = 'store_subs_fadv/'+prefix+'normal'+'_'+str(i+1)+'.txt'
            else: fn_name = 'store_subs_fadv/'+prefix+adv_type+'_'+str(i+1)+'.txt'
            try: 
                fp = open(fn_name, 'rb')
            except:
                continue
                
            signatures = pickle.load(fp)
            fp.close()
            set_of_signatures.append(signatures)

        # separate and store in for later training and evaluation 
        if adv_type == 'None': split_percentage = 0.8
        else: split_percentage = 0.8
        split_line = int(len(set_of_signatures)*split_percentage)
        
        train_set_of_signatures, test_set_of_signatures = set_of_signatures[:split_line], set_of_signatures[split_line:]
        set_of_train_dataset.append(train_set_of_signatures)
        set_of_test_dataset.append(test_set_of_signatures)
        # print(prefix, adv_type, len(set_of_signatures), len(train_set_of_signatures), len(test_set_of_signatures))
    
    one_to_nine_set_of_train_dataset.append(set_of_train_dataset)
    one_to_nine_set_of_test_dataset.append(set_of_test_dataset)    
    

In [6]:
for adv_i, adv_type in enumerate(adv_types):
    for i in range(10):
        print(adv_type, i, len(one_to_nine_set_of_train_dataset[i][adv_i]))
    print()    
    
    
prefixs = ['store_zero/', 'store_one/', 'store_two/', 'store_three/', 'store_four/', 
        'store_five/', 'store_six/', 'store_seven/', 'store_eight/', 'store_nine/']  

for idx, prefix in enumerate(prefixs):
    fn = 'store_subs_fadv/'+prefix+'nic.pt'
    
    # LOAD DATASET 
    set_of_train_dataset = one_to_nine_set_of_train_dataset[idx]
    set_of_test_dataset = one_to_nine_set_of_test_dataset[idx]

    #####################################################################
    # TRAIN
    #####################################################################
    nic = NIC()
    nic.train()
    optimizer = torch.optim.Adam(nic.parameters())
    loss_func = nn.MSELoss()
    epoches = 50

    train_accs, test_accs, losses = [], [], []
    set_train_sub_accs, set_test_sub_accs = [], []

    for epoch in range(epoches):
        total_loss = None 
        # labeling ...
        train_dataset, train_labels = [], []
        for dataset, adv_type in zip(set_of_train_dataset, adv_types):
            for singatures in dataset:
                if adv_type == 'None': 
                    for _ in range(1):
                        train_dataset.append(singatures)
                        label = torch.from_numpy(np.array([[1, 0]])).float()
                        train_labels.append(label)

                else: 
                    train_dataset.append(singatures)
                    label = torch.from_numpy(np.array([[0, 1]])).float()
                    train_labels.append(label)

        # shuffling 
        shuffle_indexs = np.arange(len(train_dataset))
        np.random.shuffle(shuffle_indexs)

        # training 
        for index in shuffle_indexs:
            singatures, label = train_dataset[index], train_labels[index]
            f1, f2, f3, f4 = preprocess(singatures)
            outputs = nic.forward(f1, f2, f3, f4)

            # for recording the training process 
            loss = loss_func(outputs, label)
            # print(round(loss.item(), 5), round(l1.item(), 5), round(l2.item(), 5))
            if total_loss is None: total_loss = loss 
            else: total_loss += loss

            # Optimization (back-propogation)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(fn)
        print('epoch:', (epoch+1), 'loss:', total_loss.item())    
        train_acc = test_guard_model(nic, set_of_train_dataset, adv_types, verbose=False)
        test_acc = test_guard_model(nic, set_of_test_dataset, adv_types, verbose=True)
        print('train (acc):', train_acc)
        print('test (acc):', test_acc)
        print()
        
#     nic.eval()
#     torch.save(nic, fn)

None 0 77
None 1 92
None 2 79
None 3 74
None 4 83
None 5 72
None 6 75
None 7 92
None 8 69
None 9 76

FGSM 0 29
FGSM 1 7
FGSM 2 47
FGSM 3 144
FGSM 4 31
FGSM 5 12
FGSM 6 6
FGSM 7 48
FGSM 8 328
FGSM 9 4

JSMA 0 90
JSMA 1 67
JSMA 2 72
JSMA 3 77
JSMA 4 80
JSMA 5 62
JSMA 6 71
JSMA 7 86
JSMA 8 118
JSMA 9 52

CWL2 0 25
CWL2 1 9
CWL2 2 79
CWL2 3 101
CWL2 4 54
CWL2 5 71
CWL2 6 50
CWL2 7 96
CWL2 8 103
CWL2 9 132

LINFPGD 0 32
LINFPGD 1 45
LINFPGD 2 77
LINFPGD 3 120
LINFPGD 4 38
LINFPGD 5 59
LINFPGD 6 35
LINFPGD 7 137
LINFPGD 8 129
LINFPGD 9 110

LINFBI 0 32
LINFBI 1 38
LINFBI 2 80
LINFBI 3 156
LINFBI 4 46
LINFBI 5 66
LINFBI 6 28
LINFBI 7 112
LINFBI 8 96
LINFBI 9 120

ENL1 0 43
ENL1 1 4
ENL1 2 76
ENL1 3 127
ENL1 4 40
ENL1 5 83
ENL1 6 33
ENL1 7 136
ENL1 8 119
ENL1 9 135

ST 0 32
ST 1 100
ST 2 71
ST 3 135
ST 4 33
ST 5 76
ST 6 24
ST 7 135
ST 8 122
ST 9 60



/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


store_subs_fadv/store_zero/nic.pt
epoch: 1 loss: 70.03255462646484
benign correct: 11 / 20
adv ( FGSM ) correct: 8 / 8
adv ( JSMA ) correct: 23 / 23
adv ( CWL2 ) correct: 6 / 7
adv ( LINFPGD ) correct: 7 / 8
adv ( LINFBI ) correct: 6 / 8
adv ( ENL1 ) correct: 10 / 11
adv ( ST ) correct: 7 / 8
acc: 0.8387096774193549
train (acc): 0.8944444444444445
test (acc): 0.8387096774193549

store_subs_fadv/store_zero/nic.pt
epoch: 2 loss: 54.84785461425781
benign correct: 12 / 20
adv ( FGSM ) correct: 8 / 8
adv ( JSMA ) correct: 23 / 23
adv ( CWL2 ) correct: 6 / 7
adv ( LINFPGD ) correct: 7 / 8
adv ( LINFBI ) correct: 7 / 8
adv ( ENL1 ) correct: 10 / 11
adv ( ST ) correct: 7 / 8
acc: 0.8602150537634409
train (acc): 0.9027777777777778
test (acc): 0.8602150537634409

store_subs_fadv/store_zero/nic.pt
epoch: 3 loss: 45.75829315185547
benign correct: 17 / 20
adv ( FGSM ) correct: 8 / 8
adv ( JSMA ) correct: 23 / 23
adv ( CWL2 ) correct: 6 / 7
adv ( LINFPGD ) correct: 7 / 8
adv ( LINFBI ) correct: 6 / 

store_subs_fadv/store_zero/nic.pt
epoch: 23 loss: 13.327866554260254
benign correct: 19 / 20
adv ( FGSM ) correct: 8 / 8
adv ( JSMA ) correct: 23 / 23
adv ( CWL2 ) correct: 6 / 7
adv ( LINFPGD ) correct: 6 / 8
adv ( LINFBI ) correct: 6 / 8
adv ( ENL1 ) correct: 10 / 11
adv ( ST ) correct: 7 / 8
acc: 0.9139784946236559
train (acc): 1.0
test (acc): 0.9139784946236559

store_subs_fadv/store_zero/nic.pt
epoch: 24 loss: 11.426095962524414
benign correct: 19 / 20
adv ( FGSM ) correct: 8 / 8
adv ( JSMA ) correct: 23 / 23
adv ( CWL2 ) correct: 6 / 7
adv ( LINFPGD ) correct: 6 / 8
adv ( LINFBI ) correct: 6 / 8
adv ( ENL1 ) correct: 10 / 11
adv ( ST ) correct: 7 / 8
acc: 0.9139784946236559
train (acc): 1.0
test (acc): 0.9139784946236559

store_subs_fadv/store_zero/nic.pt
epoch: 25 loss: 17.2197322845459
benign correct: 19 / 20
adv ( FGSM ) correct: 8 / 8
adv ( JSMA ) correct: 23 / 23
adv ( CWL2 ) correct: 6 / 7
adv ( LINFPGD ) correct: 6 / 8
adv ( LINFBI ) correct: 6 / 8
adv ( ENL1 ) correct: 10

store_subs_fadv/store_zero/nic.pt
epoch: 46 loss: 23.850717544555664
benign correct: 19 / 20
adv ( FGSM ) correct: 8 / 8
adv ( JSMA ) correct: 23 / 23
adv ( CWL2 ) correct: 6 / 7
adv ( LINFPGD ) correct: 6 / 8
adv ( LINFBI ) correct: 6 / 8
adv ( ENL1 ) correct: 10 / 11
adv ( ST ) correct: 7 / 8
acc: 0.9139784946236559
train (acc): 1.0
test (acc): 0.9139784946236559

store_subs_fadv/store_zero/nic.pt
epoch: 47 loss: 20.424814224243164
benign correct: 19 / 20
adv ( FGSM ) correct: 8 / 8
adv ( JSMA ) correct: 23 / 23
adv ( CWL2 ) correct: 6 / 7
adv ( LINFPGD ) correct: 6 / 8
adv ( LINFBI ) correct: 6 / 8
adv ( ENL1 ) correct: 10 / 11
adv ( ST ) correct: 7 / 8
acc: 0.9139784946236559
train (acc): 1.0
test (acc): 0.9139784946236559

store_subs_fadv/store_zero/nic.pt
epoch: 48 loss: 20.423198699951172
benign correct: 19 / 20
adv ( FGSM ) correct: 8 / 8
adv ( JSMA ) correct: 23 / 23
adv ( CWL2 ) correct: 6 / 7
adv ( LINFPGD ) correct: 6 / 8
adv ( LINFBI ) correct: 6 / 8
adv ( ENL1 ) correct: 

...

store_subs_fadv/store_seven/nic.pt
epoch: 49 loss: 33.42206954956055
benign correct: 0 / 23
adv ( FGSM ) correct: 12 / 12
adv ( JSMA ) correct: 22 / 22
adv ( CWL2 ) correct: 24 / 24
adv ( LINFPGD ) correct: 35 / 35
adv ( LINFBI ) correct: 28 / 28
adv ( ENL1 ) correct: 35 / 35
adv ( ST ) correct: 34 / 34
acc: 0.892018779342723
train (acc): 0.8907363420427553
test (acc): 0.892018779342723

store_subs_fadv/store_seven/nic.pt
epoch: 50 loss: 34.80457305908203
benign correct: 0 / 23
adv ( FGSM ) correct: 12 / 12
adv ( JSMA ) correct: 22 / 22
adv ( CWL2 ) correct: 24 / 24
adv ( LINFPGD ) correct: 35 / 35
adv ( LINFBI ) correct: 28 / 28
adv ( ENL1 ) correct: 35 / 35
adv ( ST ) correct: 34 / 34
acc: 0.892018779342723
train (acc): 0.8907363420427553
test (acc): 0.892018779342723

store_subs_fadv/store_eight/nic.pt
epoch: 1 loss: 82.20116424560547
benign correct: 0 / 18
adv ( FGSM ) correct: 82 / 82
adv ( JSMA ) correct: 30 / 30
adv ( CWL2 ) correct: 26 / 26
adv ( LINFPGD ) correct: 33 / 33
adv 

store_subs_fadv/store_eight/nic.pt
epoch: 20 loss: 25.052024841308594
benign correct: 0 / 18
adv ( FGSM ) correct: 82 / 82
adv ( JSMA ) correct: 30 / 30
adv ( CWL2 ) correct: 26 / 26
adv ( LINFPGD ) correct: 33 / 33
adv ( LINFBI ) correct: 25 / 25
adv ( ENL1 ) correct: 30 / 30
adv ( ST ) correct: 31 / 31
acc: 0.9345454545454546
train (acc): 0.9363468634686347
test (acc): 0.9345454545454546

store_subs_fadv/store_eight/nic.pt
epoch: 21 loss: 23.37630844116211
benign correct: 0 / 18
adv ( FGSM ) correct: 82 / 82
adv ( JSMA ) correct: 30 / 30
adv ( CWL2 ) correct: 26 / 26
adv ( LINFPGD ) correct: 33 / 33
adv ( LINFBI ) correct: 25 / 25
adv ( ENL1 ) correct: 30 / 30
adv ( ST ) correct: 31 / 31
acc: 0.9345454545454546
train (acc): 0.9363468634686347
test (acc): 0.9345454545454546

store_subs_fadv/store_eight/nic.pt
epoch: 22 loss: 25.60636329650879
benign correct: 0 / 18
adv ( FGSM ) correct: 82 / 82
adv ( JSMA ) correct: 30 / 30
adv ( CWL2 ) correct: 26 / 26
adv ( LINFPGD ) correct: 33 / 3

store_subs_fadv/store_eight/nic.pt
epoch: 41 loss: 23.644502639770508
benign correct: 0 / 18
adv ( FGSM ) correct: 82 / 82
adv ( JSMA ) correct: 30 / 30
adv ( CWL2 ) correct: 26 / 26
adv ( LINFPGD ) correct: 33 / 33
adv ( LINFBI ) correct: 25 / 25
adv ( ENL1 ) correct: 30 / 30
adv ( ST ) correct: 31 / 31
acc: 0.9345454545454546
train (acc): 0.9363468634686347
test (acc): 0.9345454545454546

store_subs_fadv/store_eight/nic.pt
epoch: 42 loss: 24.23492431640625
benign correct: 0 / 18
adv ( FGSM ) correct: 82 / 82
adv ( JSMA ) correct: 30 / 30
adv ( CWL2 ) correct: 26 / 26
adv ( LINFPGD ) correct: 33 / 33
adv ( LINFBI ) correct: 25 / 25
adv ( ENL1 ) correct: 30 / 30
adv ( ST ) correct: 31 / 31
acc: 0.9345454545454546
train (acc): 0.9363468634686347
test (acc): 0.9345454545454546

store_subs_fadv/store_eight/nic.pt
epoch: 43 loss: 21.920751571655273
benign correct: 0 / 18
adv ( FGSM ) correct: 82 / 82
adv ( JSMA ) correct: 30 / 30
adv ( CWL2 ) correct: 26 / 26
adv ( LINFPGD ) correct: 33 / 

store_subs_fadv/store_nine/nic.pt
epoch: 12 loss: 23.432817459106445
benign correct: 14 / 20
adv ( FGSM ) correct: 1 / 1
adv ( JSMA ) correct: 13 / 13
adv ( CWL2 ) correct: 34 / 34
adv ( LINFPGD ) correct: 27 / 28
adv ( LINFBI ) correct: 31 / 31
adv ( ENL1 ) correct: 34 / 34
adv ( ST ) correct: 16 / 16
acc: 0.96045197740113
train (acc): 0.9869375907111756
test (acc): 0.96045197740113

store_subs_fadv/store_nine/nic.pt
epoch: 13 loss: 24.639890670776367
benign correct: 14 / 20
adv ( FGSM ) correct: 1 / 1
adv ( JSMA ) correct: 13 / 13
adv ( CWL2 ) correct: 34 / 34
adv ( LINFPGD ) correct: 27 / 28
adv ( LINFBI ) correct: 31 / 31
adv ( ENL1 ) correct: 34 / 34
adv ( ST ) correct: 16 / 16
acc: 0.96045197740113
train (acc): 0.988388969521045
test (acc): 0.96045197740113

store_subs_fadv/store_nine/nic.pt
epoch: 14 loss: 28.078609466552734
benign correct: 13 / 20
adv ( FGSM ) correct: 1 / 1
adv ( JSMA ) correct: 13 / 13
adv ( CWL2 ) correct: 34 / 34
adv ( LINFPGD ) correct: 28 / 28
adv ( LINFB

benign correct: 13 / 20
adv ( FGSM ) correct: 1 / 1
adv ( JSMA ) correct: 13 / 13
adv ( CWL2 ) correct: 34 / 34
adv ( LINFPGD ) correct: 28 / 28
adv ( LINFBI ) correct: 31 / 31
adv ( ENL1 ) correct: 34 / 34
adv ( ST ) correct: 16 / 16
acc: 0.96045197740113
train (acc): 0.9927431059506531
test (acc): 0.96045197740113

store_subs_fadv/store_nine/nic.pt
epoch: 34 loss: 14.477971076965332
benign correct: 12 / 20
adv ( FGSM ) correct: 1 / 1
adv ( JSMA ) correct: 13 / 13
adv ( CWL2 ) correct: 34 / 34
adv ( LINFPGD ) correct: 28 / 28
adv ( LINFBI ) correct: 31 / 31
adv ( ENL1 ) correct: 34 / 34
adv ( ST ) correct: 16 / 16
acc: 0.9548022598870056
train (acc): 0.988388969521045
test (acc): 0.9548022598870056

store_subs_fadv/store_nine/nic.pt
epoch: 35 loss: 19.205293655395508
benign correct: 13 / 20
adv ( FGSM ) correct: 1 / 1
adv ( JSMA ) correct: 13 / 13
adv ( CWL2 ) correct: 34 / 34
adv ( LINFPGD ) correct: 27 / 28
adv ( LINFBI ) correct: 31 / 31
adv ( ENL1 ) correct: 34 / 34
adv ( ST ) cor